# Task 3
### Here I use the Greykite timeseries forecasting model developed by LinkedIn:
* Github: https://github.com/linkedin/greykite
* Paper 1: https://arxiv.org/pdf/2207.07788.pdf
* Paper 2: https://arxiv.org/pdf/2105.01098.pdf



### The two models developed are simple forecasting models based on time and value counts of and volume of people in the office per day.
### More sophisticated models can be developed that may use:
* Event information such as holidays
* Seasonality to account for time of year, weekly and monthly trends
* Regressors (features) that assist this prediction.

### The Mean Absolute Percentage Error for the first approach was 53.97
### For the second approach which removed outliers in the dataset the MAPE score was improved to 28.45

#### You can find prediction plots and results dataframes in the cells below.

# I found that in the 100 days prediction horizon that Tuesday will be the busiest day in the office - Result at bottom of notebook

In [ ]:
from collections import defaultdict
import warnings
import pandas as pd
warnings.filterwarnings("ignore")
import pandas as pd
import plotly
from greykite.framework.templates.autogen.forecast_config import ForecastConfig
from greykite.framework.templates.autogen.forecast_config import MetadataParam
from greykite.framework.templates.forecaster import Forecaster 
from greykite.framework.templates.model_templates import ModelTemplateEnum
from greykite.framework.utils.result_summary import summarize_grid_search_results

from greykite.algo.forecast.silverkite.constants.silverkite_holiday import SilverkiteHoliday
from greykite.framework.templates.autogen.forecast_config import ModelComponentsParam

### Initial data loading and formatting 

In [2]:
original_df = pd.read_csv('data.csv', header=0)
original_df.head()

,user.email,user.firstName,user.lastName,guest.emailAddress,guest.firstName,guest.lastName,space.name,space.type,space.floor.name,space.neighborhood.name,...,end_date,created_at,cancelled_date,first_checked_in_date,last_checked_out_date,status,status_label,building.name,building.city,building.countryName
0,ciaran.starrs+22@chargifi.com,Ciaran,Starrs,NaN,NaN,NaN,Desk 02,desk,Floor 1,emotion,...,2023-06-12T14:03:52+01:00,2023-06-12T13:59:55+01:00,NaN,2023-06-12T14:03:47+01:00,2023-06-12T14:03:52+01:00,completed,Completed - manual check-out,Bell Yard Coffee - Bell Yard,London,United Kingdom
1,ciaran.starrs+n43@kadence.co,Gerraintttt,Arlington,NaN,NaN,NaN,Desk 05,desk,Floor 1,Private Neighbourhood,...,2023-06-12T12:15:00+01:00,2023-02-06T10:59:54+00:00,2023-02-14T11:29:40+00:00,NaN,NaN,cancelled,Cancelled - other,Antrim Building,Belfast,United Kingdom
2,ciaran.starrs+22@chargifi.com,Ciaran,Starrs,NaN,NaN,NaN,Desk 02,desk,Floor 1,emotion,...,2023-06-12T11:30:51+01:00,2023-06-12T11:22:59+01:00,NaN,2023-06-12T11:30:24+01:00,2023-06-12T11:30:51+01:00,completed,Completed - manual check-out,Bell Yard Coffee - Bell Yard,London,United Kingdom
3,ciaran.starrs+22@chargifi.com,Ciaran,Starrs,NaN,NaN,NaN,Desk 04,desk,Floor 1,NaN,...,2023-06-12T10:44:42+01:00,2023-06-12T10:43:29+01:00,NaN,2023-06-12T10:44:30+01:00,2023-06-12T10:44:42+01:00,completed,Completed - manual check-out,Bell Yard Coffee - Bell Yard,London,United Kingdom
4,ciaran.starrs+22@chargifi.com,Ciaran,Starrs,NaN,NaN,NaN,Desk 03,desk,Floor 1,emotion,...,2023-06-12T10:20:56+01:00,2023-06-12T10:18:12+01:00,NaN,2023-06-12T10:19:19+01:00,2023-06-12T10:20:56+01:00,completed,Completed - manual check-out,Bell Yard Coffee - Bell Yard,London,United Kingdom


In [25]:
original_df["date_in"] = pd.to_datetime(original_df["start_date"])
original_df["date_in"] = pd.to_datetime(original_df["date_in"], utc=True).dt.date 

In [26]:
df = original_df['date_in'].value_counts().to_frame()
df.rename(columns={"date_in":"n_bookings_day"}, inplace=True)
df["date_in"] = df.index
df.reset_index(inplace=True)

df.drop("index", axis=1, inplace=True)
df

,n_bookings_day,date_in
0,2049,2022-09-12
1,2046,2022-09-23
2,448,2022-02-15
3,428,2022-02-16
4,418,2022-11-16
...,...,...
719,1,2021-04-13
720,1,2021-05-24
721,1,2021-07-30
722,1,2021-08-23


In [27]:
result = pd.merge(original_df, df, on="date_in")
result.rename(columns={"building.city":"building_city","building.name":"building_name","user.firstName":"user_firstName","space.type":"space_type"}, inplace=True)

In [28]:
result.head()

,user.email,user_firstName,user.lastName,guest.emailAddress,guest.firstName,guest.lastName,space.name,space_type,space.floor.name,space.neighborhood.name,...,cancelled_date,first_checked_in_date,last_checked_out_date,status,status_label,building_name,building_city,building.countryName,date_in,n_bookings_day
0,ciaran.starrs+22@chargifi.com,Ciaran,Starrs,NaN,NaN,NaN,Desk 02,desk,Floor 1,emotion,...,NaN,2023-06-12T14:03:47+01:00,2023-06-12T14:03:52+01:00,completed,Completed - manual check-out,Bell Yard Coffee - Bell Yard,London,United Kingdom,2023-06-12,22
1,ciaran.starrs+n43@kadence.co,Gerraintttt,Arlington,NaN,NaN,NaN,Desk 05,desk,Floor 1,Private Neighbourhood,...,2023-02-14T11:29:40+00:00,NaN,NaN,cancelled,Cancelled - other,Antrim Building,Belfast,United Kingdom,2023-06-12,22
2,ciaran.starrs+22@chargifi.com,Ciaran,Starrs,NaN,NaN,NaN,Desk 02,desk,Floor 1,emotion,...,NaN,2023-06-12T11:30:24+01:00,2023-06-12T11:30:51+01:00,completed,Completed - manual check-out,Bell Yard Coffee - Bell Yard,London,United Kingdom,2023-06-12,22
3,ciaran.starrs+22@chargifi.com,Ciaran,Starrs,NaN,NaN,NaN,Desk 04,desk,Floor 1,NaN,...,NaN,2023-06-12T10:44:30+01:00,2023-06-12T10:44:42+01:00,completed,Completed - manual check-out,Bell Yard Coffee - Bell Yard,London,United Kingdom,2023-06-12,22
4,ciaran.starrs+22@chargifi.com,Ciaran,Starrs,NaN,NaN,NaN,Desk 03,desk,Floor 1,emotion,...,NaN,2023-06-12T10:19:19+01:00,2023-06-12T10:20:56+01:00,completed,Completed - manual check-out,Bell Yard Coffee - Bell Yard,London,United Kingdom,2023-06-12,22


In [29]:
df = result[["date_in", "n_bookings_day", "building_city","building_name", "user.email", "building.countryName", 
"space_type", "user.lastName", "space.floor.name"]]

### Setup some parameters to search - this requires a more robust experimentation phase to include events, seasonality and regressors (features)

In [75]:
model_components = ModelComponentsParam(
     autoregression=None,
     uncertainty={
         "uncertainty_dict": "auto",
     }
 )

In [36]:
metadata = MetadataParam(
     time_col="date_in",  # name of the time column
     value_col="n_bookings_day",  # name of the value column
     freq="D",  
 )

## Run the main forecasting model

In [37]:
forecaster = Forecaster()
result_ts = forecaster.run_forecast_config(
     df=df,
     config=ForecastConfig(
         model_template=ModelTemplateEnum.SILVERKITE.name,
         forecast_horizon=100,  # forecasts 100 steps ahead
         coverage=0.95,  # 95% prediction intervals
         metadata_param=metadata,
         model_components_param=model_components,
    )
)

Fitting 3 folds for each of 1 candidates, totalling 3 fits


## Input data time series - Notice the huge spikes in activity. The model struggles to incorporate these.

In [38]:
ts = result_ts.timeseries
fig = ts.plot()
plotly.io.show(fig)

In [85]:
print(ts.time_stats)         # time statistics
print(ts.value_stats)        # value statistics
print(ts.freq)               # frequency
print(ts.regressor_cols)     # available regressors
#print(ts.last_date_for_fit)  # last date with value_col
print(ts.last_date_for_reg)  # last date for any regressor
print(ts.df.head())          # the standardized dataset for forecasting
print(ts.fit_df.head()) 

{'gaps':    right_before_gap right_after_gap  gap_size
0        2021-03-12      2021-03-16       3.0
1        2021-03-18      2021-03-24       5.0
2        2021-03-24      2021-03-26       1.0
3        2021-03-26      2021-03-30       3.0
4        2021-03-31      2021-04-13      12.0
5        2021-04-13      2021-04-21       7.0
6        2021-04-23      2021-04-26       2.0
7        2021-04-26      2021-04-28       1.0
8        2021-04-30      2021-05-05       4.0
9        2021-05-06      2021-05-10       3.0
10       2021-05-11      2021-05-13       1.0
11       2021-05-13      2021-05-20       6.0
12       2021-05-21      2021-05-24       2.0
13       2021-05-25      2021-05-27       1.0
14       2021-05-28      2021-05-30       1.0
15       2021-05-30      2021-06-08       8.0
16       2021-06-08      2021-06-24      15.0
17       2021-06-24      2021-07-01       6.0
18       2021-07-01      2021-07-05       3.0
19       2021-07-09      2021-07-11       1.0
20       2021-07-16      

### Here, we see how the model fits to the training data and how prediction on the test splits varies from reality

In [86]:
backtest = result_ts.backtest
fig = backtest.plot()
plotly.io.show(fig)

## Here, we have predictions from the next 100 days

In [72]:
forecast = result_ts.forecast
fig = forecast.plot()
plotly.io.show(fig)

#### Mean Absolute Percentage Error on test 53.97

In [40]:
grid_search = result_ts.grid_search
cv_results = summarize_grid_search_results(
     grid_search=grid_search,
     decimals=2,
     # The below saves space in the printed output. Remove to show all available metrics and columns.
     cv_report_metrics=None,
     column_order=["rank", "mean_test", "split_test", "mean_train", "split_train", "mean_fit_time", "mean_score_time", "params"])
 # Transposes to save space in the printed output
cv_results["params"] = cv_results["params"].astype(str)
cv_results.set_index("params", drop=True, inplace=True)
cv_results

,rank_test_MAPE,mean_test_MAPE,split_test_MAPE,mean_train_MAPE,split_train_MAPE,mean_fit_time,mean_score_time
params,,,,,,,
[],1,53.97,"(55.96, 32.66, 73.29)",105.01,"(103.76, 88.82, 122.45)",3.07,0.74


# Model 2
### Here, we remove duplicate that have the same user firstname, lastname and date.time so we can reduce the number of outliers and produce a better fit

In [56]:
original_df = original_df.drop_duplicates()
original_df.head(2)

,user.email,user.firstName,user.lastName,guest.emailAddress,guest.firstName,guest.lastName,space.name,space.type,space.floor.name,space.neighborhood.name,...,created_at,cancelled_date,first_checked_in_date,last_checked_out_date,status,status_label,building.name,building.city,building.countryName,date_in
0,ciaran.starrs+22@chargifi.com,Ciaran,Starrs,NaN,NaN,NaN,Desk 02,desk,Floor 1,emotion,...,2023-06-12T13:59:55+01:00,NaN,2023-06-12T14:03:47+01:00,2023-06-12T14:03:52+01:00,completed,Completed - manual check-out,Bell Yard Coffee - Bell Yard,London,United Kingdom,2023-06-12
1,ciaran.starrs+n43@kadence.co,Gerraintttt,Arlington,NaN,NaN,NaN,Desk 05,desk,Floor 1,Private Neighbourhood,...,2023-02-06T10:59:54+00:00,2023-02-14T11:29:40+00:00,NaN,NaN,cancelled,Cancelled - other,Antrim Building,Belfast,United Kingdom,2023-06-12


In [ ]:
df = df_2['date_in'].value_counts().to_frame()
df.rename(columns={"date_in":"n_bookings_day"}, inplace=True)
df["date_in"] = df.index
df.reset_index(inplace=True)

df.drop("index", axis=1, inplace=True)
df

In [63]:
import datetime
df_2 = pd.merge(original_df, df, on="date_in")
df_2['name'] = df_2.apply(lambda row: row['user.firstName'] + row["user.lastName"], axis=1)
df_2 = df_2.drop_duplicates(subset=["date_in", "name"])
df_2[df_2["date_in"] == datetime.date(2022, 9, 12)].head()

,user.email,user.firstName,user.lastName,guest.emailAddress,guest.firstName,guest.lastName,space.name,space.type,space.floor.name,space.neighborhood.name,...,first_checked_in_date,last_checked_out_date,status,status_label,building.name,building.city,building.countryName,date_in,n_bookings_day,name
27641,ciaran.starrs+22@chargifi.com,Ciaran,Starrs,NaN,NaN,NaN,Desk 104,desk,500 desks on this floor,NaN,...,NaN,NaN,cancelled,Cancelled by user,Large number of desks on floors (do not delete),Peterborough,United Kingdom,2022-09-12,11,CiaranStarrs
27644,ciaran.starrs+n45@kadence.co,Dax,Bamberger,NaN,NaN,NaN,Desk 03,desk,Floor 1,emotion,...,NaN,NaN,no-check-in,No check-in,Bell Yard Coffee - Bell Yard,London,United Kingdom,2022-09-12,11,DaxBamberger
27645,ryan+demo@chargifi.com,Ryan,Demo,NaN,NaN,NaN,Desk 0001,desk,Big floor,NaN,...,NaN,NaN,no-check-in,No check-in,Ryan Building for bulk booking tests,Salisbury,United Kingdom,2022-09-12,11,RyanDemo
29647,ciaran.starrs+sv@kadence.co,Stig,Bjarmabie,NaN,NaN,NaN,Desk 02,desk,Floor 1,NaN,...,NaN,NaN,cancelled,Cancelled - other,Antrim Building,Belfast,United Kingdom,2022-09-12,11,StigBjarmabie
29648,NaN,NaN,NaN,NaN,NaN,NaN,Desk 02,desk,Floor 1,NaN,...,NaN,NaN,cancelled,Cancelled - other,Antrim Building,Belfast,United Kingdom,2022-09-12,11,NaN


In [64]:
df_2["date_in"].value_counts()

2023-02-21    78
2023-02-23    78
2023-02-22    78
2023-02-20    55
2023-02-17    55
              ..
2021-09-05     1
2021-03-31     1
2021-03-24     1
2021-05-13     1
2021-08-30     1
Name: date_in, Length: 724, dtype: int64

## Run Model 2

In [70]:
forecaster = Forecaster()
result_ts = forecaster.run_forecast_config(
     df=df_2,
     config=ForecastConfig(
         model_template=ModelTemplateEnum.SILVERKITE.name,
         forecast_horizon=100,  # forecasts 100 steps ahead
         coverage=0.95,  # 95% prediction intervals
         metadata_param=metadata,
         model_components_param=model_components,
    )
)

Fitting 3 folds for each of 1 candidates, totalling 3 fits


### Clearly, the input timeseries is much more consistent

In [71]:
ts = result_ts.timeseries
fig = ts.plot()
plotly.io.show(fig)

# Predictions against the test set are much more closely aligned, resulting in an improved MAPE score

In [72]:
backtest = result_ts.backtest
fig = backtest.plot()
plotly.io.show(fig)

## Final predictions

In [73]:
forecast = result_ts.forecast
fig = forecast.plot()
plotly.io.show(fig)

In [74]:
grid_search = result_ts.grid_search
cv_results = summarize_grid_search_results(
     grid_search=grid_search,
     decimals=2,
     # The below saves space in the printed output. Remove to show all available metrics and columns.
     cv_report_metrics=None,
     column_order=["rank", "mean_test", "split_test", "mean_train", "split_train", "mean_fit_time", "mean_score_time", "params"])
 # Transposes to save space in the printed output
cv_results["params"] = cv_results["params"].astype(str)
cv_results.set_index("params", drop=True, inplace=True)
cv_results.transpose()

params,[]
rank_test_MAPE,1
mean_test_MAPE,28.45
split_test_MAPE,"(33.89, 25.69, 25.76)"
mean_train_MAPE,27.93
split_train_MAPE,"(33.47, 25.87, 24.46)"
mean_fit_time,3.88
mean_score_time,1.15


### Use the results dataframe to find the busiest day

In [95]:
forecast_df = forecast.__dict__['df'][['date_in','forecast']].iloc[-100:]
forecast_df

,date_in,forecast
823,2023-06-13,22.750973
824,2023-06-14,23.609052
825,2023-06-15,23.156981
826,2023-06-16,23.450923
827,2023-06-17,13.357826
...,...,...
918,2023-09-16,17.142967
919,2023-09-17,14.847092
920,2023-09-18,25.315023
921,2023-09-19,28.089701


In [99]:
forecast_df['day'] = forecast_df['date_in'].dt.dayofweek
forecast_df

,date_in,forecast,day
823,2023-06-13,22.750973,1
824,2023-06-14,23.609052,2
825,2023-06-15,23.156981,3
826,2023-06-16,23.450923,4
827,2023-06-17,13.357826,5
...,...,...,...
918,2023-09-16,17.142967,5
919,2023-09-17,14.847092,6
920,2023-09-18,25.315023,0
921,2023-09-19,28.089701,1


In [100]:
dff = forecast_df.groupby(["day"]).forecast.sum().reset_index()
dff

,day,forecast
0,0,322.288739
1,1,380.134684
2,2,392.048374
3,3,358.505751
4,4,370.475271
5,5,215.023953
6,6,186.395779


In [103]:
print(f"Day {dff['forecast'].idxmax()} is max with predicted total of {dff['forecast'].max()}")

Day 2 is max with predicted total of 392.04837402652026
